# Tutorial 4: Turbine Assembly

Here's what we've done so far in these tutotirals:

+ Ran two simple cost models of turbines. In these, we estiamted masses of components and cost per kilogram of those components.

+ We learned how OpenMDAO makes *components* when we calculated the Betz limit by modelling an idealized `ActuatorDisc` as a subclass of `ExplicitComponent`.

+ We learned how to group multiple components into groups with the OpenMDAO `Group` class when we modelled the Sellar problem.

We can now turn our attention back to WISDEM and put together a rotor, drivetrain and tower to model a complete wind turbine. We will use the tools we have gained so far in these tutorials to accomplish this.

This is a significant increase in complexity from our previous toy examples. This tutorial doesn't aim to give an exhaustive line-by-line explanation of nearly 400 lines of source code. However, these fundamental building blocks of components, groups and susbsytems are used to model systems of significant complexity.

## First, we need to import our dependencies

There are many dependencies we need to import. Of key interesst to use here are various parts of WISDEM that we will assemble into our model.

```python
from wisdem.rotorse.rotor import RotorSE, Init_RotorSE_wRefBlade
from wisdem.rotorse.rotor_geometry_yaml import ReferenceBlade
from wisdem.towerse.tower import TowerSE
from wisdem.commonse import NFREQ
from wisdem.commonse.environment import PowerWind, LogWind
from wisdem.commonse.turbine_constraints import TurbineConstraints
from wisdem.turbine_costsse.turbine_costsse_2015 import Turbine_CostsSE_2015
from wisdem.plant_financese.plant_finance import PlantFinance
from wisdem.drivetrainse.drivese_omdao import DriveSE
```

In [1]:
from __future__ import print_function
import numpy as np
from pprint import pprint
from openmdao.api import IndepVarComp, ExplicitComponent, Group, Problem, ScipyOptimizeDriver, SqliteRecorder, NonlinearRunOnce, DirectSolver
try:
    from openmdao.api import pyOptSparseDriver
except:
    pass
from wisdem.rotorse.rotor import RotorSE, Init_RotorSE_wRefBlade
from wisdem.rotorse.rotor_geometry_yaml import ReferenceBlade
from wisdem.towerse.tower import TowerSE
from wisdem.commonse import NFREQ
from wisdem.commonse.environment import PowerWind, LogWind
from wisdem.commonse.turbine_constraints import TurbineConstraints
from wisdem.turbine_costsse.turbine_costsse_2015 import Turbine_CostsSE_2015
from wisdem.plant_financese.plant_finance import PlantFinance
from wisdem.drivetrainse.drivese_omdao import DriveSE

from wisdem.commonse.mpi_tools import MPI

After we import our libraries, we make a subclass of `Group` which will hold our assembly of the turbine.

For our assembly, we will first need some independent variables. For this we will use `IndepVarComp`. Remember that `IndepVarComp` creates *outputs* that send the variables to the *inputs* of other subsystems in the same group. At the bottom of this section, note the `promotes=['*']` which makes these variables available to other subsystems.

```python
myIndeps = IndepVarComp()
myIndeps.add_discrete_output('crane',    False)

# Turbine Costs
myIndeps.add_discrete_output('bearing_number', 0)

# Tower and Frame3DD options
myIndeps.add_output('project_lifetime',             0.0, units='yr')
myIndeps.add_output('max_taper_ratio',              0.0)
myIndeps.add_output('min_diameter_thickness_ratio', 0.0)

# Environment
myIndeps.add_output('wind_bottom_height',   0.0, units='m')
myIndeps.add_output('wind_beta',            0.0, units='deg')
myIndeps.add_output('cd_usr', -1.)

# Design standards
myIndeps.add_output('gamma_b', 0.0)
myIndeps.add_output('gamma_n', 0.0)

# RNA
myIndeps.add_discrete_output('rna_weightM', True)

# Column
myIndeps.add_output('morison_mass_coefficient', 2.0)
myIndeps.add_output('material_density',         0.0, units='kg/m**3')
myIndeps.add_output('E',                        0.0, units='N/m**2')
myIndeps.add_output('yield_stress',             0.0, units='N/m**2')

# Pontoons
myIndeps.add_output('G', 0.0, units='N/m**2')

# LCOE
myIndeps.add_output('labor_cost_rate',      0.0, units='USD/min')
myIndeps.add_output('material_cost_rate',   0.0, units='USD/kg')
myIndeps.add_output('painting_cost_rate',   0.0, units='USD/m**2')
myIndeps.add_discrete_output('number_of_turbines', 0)
myIndeps.add_output('annual_opex',          0.0, units='USD/kW/yr') # TODO: Replace with output connection
myIndeps.add_output('bos_costs',            0.0, units='USD/kW') # TODO: Replace with output connection
myIndeps.add_output('fixed_charge_rate',    0.0)
myIndeps.add_output('wake_loss_factor',     0.0)

self.add_subsystem('myIndeps', myIndeps, promotes=['*'])
```

Then we add a `RotorSE` group that models our rotor. Note the `promotes=['*']` again. This makes all of its variables available to the other subsystems for implicit connection to other variables of the same name. We attach this rotor as a subsystem in our group. This subsystem is called `rotorse`.

```python
self.add_subsystem('rotorse', RotorSE(RefBlade=RefBlade,
                                              npts_coarse_power_curve=20,
                                              npts_spline_power_curve=200,
                                              regulation_reg_II5=True,
                                              regulation_reg_III=False,
                                              Analysis_Level=Analysis_Level,
                                              FASTpref=self.options['FASTpref'],
                                              topLevelFlag=True), promotes=['*'])
```

After that we add a `DriveSE` group which models our drivetrain. For this group, we do not use `promotes=['*']`; rather, we explicitly promote the variables we want available to the other subsystems. This subsystem is called `drive`.

```python
self.add_subsystem('drive', DriveSE(debug=False,
                                            number_of_main_bearings=1,
                                            topLevelFlag=False),
                           promotes=['machine_rating', 'overhang',
                           'hub_mass','bedplate_mass','gearbox_mass','generator_mass','hss_mass','hvac_mass','lss_mass','cover_mass',
                           'pitch_system_mass','platforms_mass','spinner_mass','transformer_mass','vs_electronics_mass','yaw_mass'])
```

As our last step of the turbine assembly, we add a `TowerSE` group which models our tower. Again, we explicitly list the variables we want to promote from this group. This subsystem is called `tow`.

```python
self.add_subsystem('tow', TowerSE(nLC=1,
                                 nPoints=Nsection_Tow+1,
                                 nFull=5*Nsection_Tow+1,
                                 wind='PowerWind',
                                 topLevelFlag=False),
                   promotes=['water_density','water_viscosity','wave_beta',
                             'significant_wave_height','significant_wave_period',
                             'material_density','E','G','tower_section_height',
                             'tower_wall_thickness', 'tower_outer_diameter',
                             'tower_outfitting_factor','tower_buckling_length',
                             'max_taper','min_d_to_t','rna_mass','rna_cg','rna_I',
                             'tower_mass','tower_I_base','hub_height',
                             'foundation_height','monopile','soil_G','soil_nu',
                             'suctionpile_depth','gamma_f','gamma_m','gamma_b','gamma_n','gamma_fatigue',
                             'labor_cost_rate','material_cost_rate','painting_cost_rate','z_full','d_full','t_full',
                             'DC','shear','geom','tower_force_discretization','nM','Mmethod','lump','tol','shift'])
```

The `tow`, `drive` and `rotorse` subsystems comprise our turbine. But we need to place a tip clearance constraint on this assembly to ensure that the tips of the rotor do not collide with the tower. The `TurbineConstraints` group enforces this constraint. We'll add this as a subsystem called `tcons`.

```python
self.add_subsystem('tcons', TurbineConstraints(nFull=5*Nsection_Tow+1), promotes=['*'])
```

Recall our simple cost model for the turbine. Let's add that as the subsystem `tcost`.

```python
self.add_subsystem('tcost', Turbine_CostsSE_2015(verbosity=self.options['VerbosityCosts'], topLevelFlag=False), promotes=['*'])
```

Finally, we want to calculate the LCOE of the turbine.

```python
self.add_subsystem('plantfinancese', PlantFinance(verbosity=self.options['VerbosityCosts']), promotes=['machine_rating', 'lcoe'])
```

OpenMDAO needs to be told how to connect the components that do use `promotes=['*']`. For exmaple, for `DriveSE` there a couple of variables that are connected this way:

```python
# Connections to DriveSE
self.connect('diameter',        'drive.rotor_diameter')     
self.connect('rated_Q',         'drive.rotor_torque')
```

In these lines, OpenMDAO finds the `diameter` variable promoted into the group's namespace and connects it to the `rotor_diameter` input in the subsystem `drive`. This is addressed as `drive.rotor_diameter`. Similarly `rated_Q` is connected `driver.rotor_torque`. Such connections are good way to "wire" variables with different names together. There are many such connections in our code.


With these subsystems, we can make our group as shown below.

In [2]:
# Group to link the openmdao components
class LandBasedTurbine(Group):

    def initialize(self):
        self.options.declare('RefBlade')
        self.options.declare('FASTpref', default={})
        self.options.declare('Nsection_Tow', default = 6)
        self.options.declare('VerbosityCosts', default = True)
        
        
    def setup(self):
        
        RefBlade     = self.options['RefBlade']
        Nsection_Tow = self.options['Nsection_Tow']        
        if 'Analysis_Level' in self.options['FASTpref']:
            Analysis_Level = self.options['FASTpref']['Analysis_Level']
        else:
            Analysis_Level = 0
        
        
        # Define all input variables from all models
        myIndeps = IndepVarComp()
        myIndeps.add_discrete_output('crane',    False)

        # Turbine Costs
        myIndeps.add_discrete_output('bearing_number', 0)
        
        # Tower and Frame3DD options
        myIndeps.add_output('project_lifetime',             0.0, units='yr')
        myIndeps.add_output('max_taper_ratio',              0.0)
        myIndeps.add_output('min_diameter_thickness_ratio', 0.0)

        # Environment
        myIndeps.add_output('wind_bottom_height',   0.0, units='m')
        myIndeps.add_output('wind_beta',            0.0, units='deg')
        myIndeps.add_output('cd_usr', -1.)

        # Design standards
        myIndeps.add_output('gamma_b', 0.0)
        myIndeps.add_output('gamma_n', 0.0)

        # RNA
        myIndeps.add_discrete_output('rna_weightM', True)
        
        # Column
        myIndeps.add_output('morison_mass_coefficient', 2.0)
        myIndeps.add_output('material_density',         0.0, units='kg/m**3')
        myIndeps.add_output('E',                        0.0, units='N/m**2')
        myIndeps.add_output('yield_stress',             0.0, units='N/m**2')

        # Pontoons
        myIndeps.add_output('G', 0.0, units='N/m**2')
        
        # LCOE
        myIndeps.add_output('labor_cost_rate',      0.0, units='USD/min')
        myIndeps.add_output('material_cost_rate',   0.0, units='USD/kg')
        myIndeps.add_output('painting_cost_rate',   0.0, units='USD/m**2')
        myIndeps.add_discrete_output('number_of_turbines', 0)
        myIndeps.add_output('annual_opex',          0.0, units='USD/kW/yr') # TODO: Replace with output connection
        myIndeps.add_output('bos_costs',            0.0, units='USD/kW') # TODO: Replace with output connection
        myIndeps.add_output('fixed_charge_rate',    0.0)
        myIndeps.add_output('wake_loss_factor',     0.0)
        
        self.add_subsystem('myIndeps', myIndeps, promotes=['*'])

        
        # Add components
        self.add_subsystem('rotorse', RotorSE(RefBlade=RefBlade,
                                              npts_coarse_power_curve=20,
                                              npts_spline_power_curve=200,
                                              regulation_reg_II5=True,
                                              regulation_reg_III=False,
                                              Analysis_Level=Analysis_Level,
                                              FASTpref=self.options['FASTpref'],
                                              topLevelFlag=True), promotes=['*'])
        
        self.add_subsystem('drive', DriveSE(debug=False,
                                            number_of_main_bearings=1,
                                            topLevelFlag=False),
                           promotes=['machine_rating', 'overhang',
                                     'hub_mass','bedplate_mass','gearbox_mass','generator_mass','hss_mass','hvac_mass','lss_mass','cover_mass',
                                     'pitch_system_mass','platforms_mass','spinner_mass','transformer_mass','vs_electronics_mass','yaw_mass'])
        
        # Tower and substructure
        self.add_subsystem('tow',TowerSE(nLC=1,
                                         nPoints=Nsection_Tow+1,
                                         nFull=5*Nsection_Tow+1,
                                         wind='PowerWind',
                                         topLevelFlag=False),
                           promotes=['water_density','water_viscosity','wave_beta',
                                     'significant_wave_height','significant_wave_period',
                                     'material_density','E','G','tower_section_height',
                                     'tower_wall_thickness', 'tower_outer_diameter',
                                     'tower_outfitting_factor','tower_buckling_length',
                                     'max_taper','min_d_to_t','rna_mass','rna_cg','rna_I',
                                     'tower_mass','tower_I_base','hub_height',
                                     'foundation_height','monopile','soil_G','soil_nu',
                                     'suctionpile_depth','gamma_f','gamma_m','gamma_b','gamma_n','gamma_fatigue',
                                     'labor_cost_rate','material_cost_rate','painting_cost_rate','z_full','d_full','t_full',
                                     'DC','shear','geom','tower_force_discretization','nM','Mmethod','lump','tol','shift'])

        # Turbine constraints
        self.add_subsystem('tcons', TurbineConstraints(nFull=5*Nsection_Tow+1), promotes=['*'])
        
        # Turbine costs
        self.add_subsystem('tcost', Turbine_CostsSE_2015(verbosity=self.options['VerbosityCosts'], topLevelFlag=False), promotes=['*'])

        # LCOE Calculation
        self.add_subsystem('plantfinancese', PlantFinance(verbosity=self.options['VerbosityCosts']), promotes=['machine_rating','lcoe'])
        
    
        # Set up connections

        # Connections to DriveSE
        self.connect('diameter',        'drive.rotor_diameter')     
        self.connect('rated_Q',         'drive.rotor_torque')
        self.connect('rated_Omega',     'drive.rotor_rpm')
        self.connect('Fxyz_total',      'drive.Fxyz')
        self.connect('Mxyz_total',      'drive.Mxyz')
        self.connect('I_all_blades',        'drive.blades_I')
        self.connect('mass_one_blade',  'drive.blade_mass')
        self.connect('chord',           'drive.blade_root_diameter', src_indices=[0])
        self.connect('Rtip',            'drive.blade_length', src_indices=[0])
        self.connect('drivetrainEff',   'drive.drivetrain_efficiency', src_indices=[0])
        self.connect('tower_outer_diameter', 'drive.tower_top_diameter', src_indices=[-1])

        self.connect('material_density', 'tow.tower.rho')

        # Connections to TowerSE
        self.connect('drive.top_F',         'tow.pre.rna_F')
        self.connect('drive.top_M',         'tow.pre.rna_M')
        self.connect('drive.rna_I_TT',            ['rna_I','tow.pre.mI'])
        self.connect('drive.rna_cm',              ['rna_cg','tow.pre.mrho'])
        self.connect('drive.rna_mass',            ['rna_mass','tow.pre.mass'])
        self.connect('rs.gust.V_gust',          'tow.wind.Uref')
        self.connect('wind_reference_height',   ['tow.wind.zref','wind.zref'])
        # self.connect('wind_bottom_height',      ['tow.wind.z0','tow.wave.z_surface', 'wind.z0'])  # offshore
        self.connect('wind_bottom_height',      ['tow.wind.z0', 'wind.z0'])
        self.connect('shearExp',                ['tow.wind.shearExp'])
        # self.connect('morison_mass_coefficient','tow.cm')                                         # offshore
        self.connect('yield_stress',            'tow.sigma_y')
        self.connect('max_taper_ratio',         'max_taper')
        self.connect('min_diameter_thickness_ratio', 'min_d_to_t')

        self.connect('rho',         'tow.windLoads.rho')
        self.connect('mu',          'tow.windLoads.mu')
        self.connect('wind_beta',   'tow.windLoads.beta')
        
        # Connections to TurbineConstraints
        self.connect('nBlades',                 ['blade_number', 'drive.number_of_blades'])
        self.connect('control_maxOmega',        'rotor_omega')
        self.connect('tow.post.structural_frequencies', 'tower_freq')        
                
        # Connections to TurbineCostSE
        self.connect('mass_one_blade',              'blade_mass')
        self.connect('drive.mainBearing.mb_mass',   'main_bearing_mass')
        self.connect('total_blade_cost',            'blade_cost_external')
        
        # Connections to PlantFinanceSE
        self.connect('AEP',                 'plantfinancese.turbine_aep')
        self.connect('turbine_cost_kW',     'plantfinancese.tcc_per_kW')
        self.connect('number_of_turbines',  'plantfinancese.turbine_number')
        self.connect('bos_costs',           'plantfinancese.bos_per_kW')
        self.connect('annual_opex',         'plantfinancese.opex_per_kW')

That's a lot of variables to connect! So let's make a function that can do this for us. Let's look at a few key lines of code for what is in this function:

First, we initialize parameters for our rotor model.
```python
prob = Init_RotorSE_wRefBlade(prob, blade, Analysis_Level = Analysis_Level, fst_vt = fst_vt)
```

Then there are a number of lines similar to the following:

```python
prob['material_density'] = 7850.0
```

Remember the `myIndeps.add_output('material_density', 0.0, units='kg/m**3')` line above? This is where we are providing the value for that independent variable.

In [3]:
def Init_LandBasedAssembly(prob, blade, Nsection_Tow, Analysis_Level = 0, fst_vt = {}):

    prob = Init_RotorSE_wRefBlade(prob, blade, Analysis_Level = Analysis_Level, fst_vt = fst_vt)
    
    
    # Environmental parameters for the tower
    # prob['wind_reference_speed']           = 11.0
    prob['wind_reference_height']          = prob['hub_height']

    # Steel properties for the tower
    prob['material_density']               = 7850.0
    prob['E']                              = 200e9
    prob['G']                              = 79.3e9
    prob['yield_stress']                   = 3.45e8

    # Design constraints
    prob['max_taper_ratio']                = 0.4
    prob['min_diameter_thickness_ratio']   = 120.0

    # Safety factors
    prob['gamma_fatigue']   = 1.755 # (Float): safety factor for fatigue
    prob['gamma_f']         = 1.35  # (Float): safety factor for loads/stresses
    prob['gamma_m']         = 1.3   # (Float): safety factor for materials
    prob['gamma_freq']      = 1.1   # (Float): safety factor for resonant frequencies
    prob['gamma_n']         = 1.0
    prob['gamma_b']         = 1.1
    
    # Tower
    prob['foundation_height']              = 0.0 #-prob['water_depth']
    # prob['tower_outer_diameter']           = np.linspace(10.0, 3.87, Nsection_Tow+1)
    prob['tower_outer_diameter']           = np.linspace(6.0, 3.87, Nsection_Tow+1)
    prob['tower_section_height']           = (prob['hub_height'] - prob['foundation_height']) / Nsection_Tow * np.ones(Nsection_Tow)
    prob['tower_wall_thickness']           = np.linspace(0.027, 0.019, Nsection_Tow)
    prob['tower_buckling_length']          = 30.0
    prob['tower_outfitting_factor']        = 1.07

    prob['DC']      = 80.0
    prob['shear']   = True
    prob['geom']    = False
    prob['tower_force_discretization'] = 5.0
    prob['nM']      = 2
    prob['Mmethod'] = 1
    prob['lump']    = 0
    prob['tol']     = 1e-9
    prob['shift']   = 0.0
    
    # Plant size
    prob['project_lifetime'] = prob['lifetime'] = 20.0    
    prob['number_of_turbines']             = 200. * 1.e+006 / prob['machine_rating']
    prob['annual_opex']                    = 43.56 # $/kW/yr
    prob['bos_costs']                      = 517.0 # $/kW
    
    # For RNA
    prob['rna_weightM'] = True

    # For turbine costs
    # prob['offshore']             = False
    prob['crane']                = False
    prob['bearing_number']       = 2
    prob['crane_cost']           = 0.0
    prob['labor_cost_rate']      = 3.0
    prob['material_cost_rate']   = 2.0
    prob['painting_cost_rate']   = 28.8
    
    # Gearbox
    prob['drive.gear_ratio']        = 96.76  # 97:1 as listed in the 5 MW reference document
    prob['drive.shaft_angle']       = prob['tilt']*np.pi / 180.0  # rad
    prob['drive.shaft_ratio']       = 0.10
    prob['drive.planet_numbers']    = [3, 3, 1]
    prob['drive.shrink_disc_mass']  = 333.3 * prob['machine_rating'] / 1e6  # estimated
    prob['drive.carrier_mass']      = 8000.0  # estimated
    prob['drive.flange_length']     = 0.5
    prob['overhang']                = 5.0
    prob['drive.distance_hub2mb']   = 1.912  # length from hub center to main bearing, leave zero if unknown
    prob['drive.gearbox_input_xcm'] = 0.1
    prob['drive.hss_input_length']  = 1.5
    prob['drive.yaw_motors_number'] = 1
    
    return prob

What do we have so far?

1. A group of subsystems that model our rotor, drivetrain and tower

1. A function which sets the values of our independent variables needed by that group.

What remains is to set up our problem to optimize our group as initialized by our function. Again, here are some highlights of the code below:

```python
optFlag = False
```

If this is set to `True`, the model will run repeatedly to optimize the problem, eventually outputting the design variables. But for the purposes of a quick demo, we can set this to `False`. A true optimization would take a long time.

```python
# Reference rotor design
fname_schema          = "../wisdem/rotorse/turbine_inputs/IEAontology_schema.yaml"
fname_input           = "../wisdem/rotorse/turbine_inputs/nrel5mw_mod_update.yaml"

Analysis_Level        = 0 # 0: Run CCBlade; 1: Update FAST model at each iteration but do not run; 2: Run FAST w/ ElastoDyn; 3: (Not implemented) Run FAST w/ BeamDyn
# Initialize blade design
refBlade = ReferenceBlade()
refBlade.verbose      = True
refBlade.NINPUT       = 8
refBlade.NPTS         = 50
refBlade.spar_var     = ['Spar_Cap_SS', 'Spar_Cap_PS'] # SS, then PS
refBlade.te_var       = 'TE_reinforcement'
refBlade.validate     = False
refBlade.fname_schema = fname_schema
blade = refBlade.initialize(fname_input)
# Initialize tower design
Nsection_Tow = 6
```

These lines set up the blade design we will use in our rotor. The `.yaml` contain the specifications of the blades in our rotor. YAML is a format that makes writing these specifications more convenient than writing them in Python. We also set up the number of sections in the tower here.

```python
prob = Problem()
prob.model = LandBasedTurbine(RefBlade=blade, Nsection_Tow=Nsection_Tow, VerbosityCosts=True)
prob.setup()

prob = Init_LandBasedAssembly(prob, blade, Nsection_Tow)
prob.model.nonlinear_solver = NonlinearRunOnce()
prob.model.linear_solver = DirectSolver()
```

Here, we create a problem and set its model to be our `LandBasedTurbine` we created above. After we do that, we pass our problm to the `Init_LandBasedAssembly()` function which takes our original problem and uses it to create another problem. After that we set solvers on our new `Problem` we can run it with

```python
prob.run_driver()
```

In [6]:
# Reference rotor design
fname_schema          = "../wisdem/rotorse/turbine_inputs/IEAontology_schema.yaml"
fname_input           = "../wisdem/rotorse/turbine_inputs/nrel5mw_mod_update.yaml"

Analysis_Level        = 0 # 0: Run CCBlade; 1: Update FAST model at each iteration but do not run; 2: Run FAST w/ ElastoDyn; 3: (Not implemented) Run FAST w/ BeamDyn
# Initialize blade design
refBlade = ReferenceBlade()
refBlade.verbose      = True
refBlade.NINPUT       = 8
refBlade.NPTS         = 50
refBlade.spar_var     = ['Spar_Cap_SS', 'Spar_Cap_PS'] # SS, then PS
refBlade.te_var       = 'TE_reinforcement'
refBlade.validate     = False
refBlade.fname_schema = fname_schema
blade = refBlade.initialize(fname_input)
# Initialize tower design
Nsection_Tow = 6

# Create a problem for our LandBasedTurbine
prob = Problem()
prob.model=LandBasedTurbine(RefBlade=blade, Nsection_Tow=Nsection_Tow, VerbosityCosts=True)
prob.setup()

prob = Init_LandBasedAssembly(prob, blade, Nsection_Tow)
prob.model.nonlinear_solver = NonlinearRunOnce()
prob.model.linear_solver = DirectSolver()

prob.model.approx_totals()

prob.run_driver()

Running initialization: ../wisdem/rotorse/turbine_inputs/nrel5mw_mod_update.yaml
Complete: Load Input File: 	0.007081 s
Complete: Geometry Analysis: 	0.360268 s
Complete: Precomp Conversion: 	0.165845 s
################################################
Computation of costs of the main turbine components from TurbineCostSE
Blade cost              212.362 k USD       mass 16122.567 kg
Pitch system cost       235.317 k USD       mass 10647.806 kg
Hub cost                41.540 k USD       mass 10651.208 kg
Spinner cost            14.560 k USD       mass 1311.715 kg
------------------------------------------------
Rotor cost              928.502 k USD       mass 70978.430 kg

LSS cost                168.179 k USD       mass 14132.675 kg
Main bearing cost       21.893 k USD       mass 2432.512 kg
Gearbox cost            530.723 k USD       mass 41141.295 kg
HSS cost                11.529 k USD       mass 1695.431 kg
Generator cost          207.078 k USD       mass 16699.851 kg
Bedplate cost 

False

Here we see the output from the `TurbineCostSE` and `Plant_finance_SE`. (Don't worry about the `False`)